In [6]:
# def load_data(filename):
#     with open(filename, 'r', encoding='utf-8') as file:
#         data = file.read().strip().split('\n')  # Splitting data by double newlines

#     formatted_data = []
#     for record in data:
#         text, entities = '', []
#         lines = record.split('\n')
#         start = 0  # Initialize start of the first token
#         entity_start = None  # Initialize the start of an entity
#         entity_type = None  # Initialize the entity type
#         for line in lines:
#             parts = line.split(',')
#             if len(parts) >= 2:  # Ensure there's at least a token and a tag
#                 token, tag = parts[0], parts[1]
#                 # Update text with token followed by space
#                 text += token + ' '
#                 end = start + len(token)  # End index of the current token
#                 if tag.startswith('B-') or tag.startswith('I-'):  # Check if the tag indicates an entity
#                     if tag.startswith('B-'):
#                         # If there was a previous entity, append it
#                         if entity_start is not None:
#                             entities.append((entity_start, prev_end, entity_type))
#                         # Start new entity
#                         entity_start = start
#                         entity_type = tag.split('-')[1]
#                     if tag.startswith('I-') and entity_start is not None:
#                         # Continue the entity
#                         entity_type = tag.split('-')[1]  # Update the type if needed (typically not necessary)
#                 else:
#                     # If ending part of entity, append it
#                     if entity_start is not None:
#                         entities.append((entity_start, end, entity_type))
#                         entity_start = None  # Reset for the next entity

#                 # Prepare for next token
#                 start = end + 1
#                 prev_end = end  # Remember the end of the last token

#         # Check for any trailing entity at the end
#         if entity_start is not None:
#             entities.append((entity_start, prev_end, entity_type))

#         # Append the parsed data if there are entities
#         if entities:
#             formatted_data.append((text.strip(), {'entities': entities}))

#     return formatted_data

def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n\n')  # Split records by double newlines
    print(f'Number of total sentences: {len(data)}')
    formatted_data = []
    for record in data:
        text, entities = '', []
        lines = record.split('\n')
        start = 0  # Initialize start index of the first token
        entity_start = None  # Initialize the start index of an entity
        entity_type = None  # Initialize the entity type
        prev_end = 0  # Initialize end index of the previous token

        for line in lines:
            parts = line.split('\t')
            if len(parts) >= 2:
                token, tag = parts[0], parts[1]
                text += token + ' '
                end = start + len(token)
                
                if tag.startswith('B-'):
                    if entity_start is not None:
                        entities.append((entity_start, prev_end, entity_type))
                    entity_start = start
                    entity_type = tag.split('-')[1]

                elif tag.startswith('I-'):
                    
                    if entity_start is not None:
#                         print(tag, entity_start)
                        if entity_type != tag.split('-')[1]:
                            # Append the previous entity and start a new one if types mismatch
                            entities.append((entity_start, prev_end, entity_type))
                            entity_start = start
                            entity_type = tag.split('-')[1]
                            print((entity_start, prev_end, entity_type))

                else:
                    if entity_start is not None:
                        entities.append((entity_start, prev_end, entity_type))
                        entity_start = None

                start = end + 1
                prev_end = end

        if entity_start is not None:
            entities.append((entity_start, prev_end, entity_type))

        if entities:
            formatted_data.append((text.strip(), {'entities': entities}))
        else:
            formatted_data.append((text.strip(), {'entities': []}))  # Include text without entities for context

    return formatted_data


# # Load the data
train_data = load_data('data/final/splits/train.csv')
dev_data = load_data('data/final/splits/dev.csv')
test_data = load_data('data/final/splits/test.csv')


# Load the data
# train_data = load_data('data/ner_multi/splits/train_correct.csv')
# dev_data = load_data('data/ner_multi/splits/dev_correct.csv')
# test_data = load_data('data/ner_multi/splits/test_correct.csv')


Number of total sentences: 2510
Number of total sentences: 313
Number of total sentences: 315


In [7]:
train_data

[('Категорія .', {'entities': []}),
 ('З розбивкою на слова та відновленням втрачених фрагментів текст виглядає таким чином : Помени [ П]роко[пиꙗ ] спасѵ твое[го р]аба .',
  {'entities': []}),
 ('Друга чверть XІ – кінець XVІ ст. ; за формою прорізів .', {'entities': []}),
 ('Поверхня стіни в місці розташування напису сильно пошкоджена ви- боями , внаслідок чого більша частина складових знищена : г͠\ue01fпо--ꙁ\ue01fраб- | своемум -- ъку-- | оу\ue01fп--ѧн\ue01f--\ue01f',
  {'entities': []}),
 ('З розбивкою на слова текст виглядає таким чином : С(вѧ)тии м(оу)ч(е)н(и)ци молите Б(ог)а за мѧ г{рѣшьнаго } .',
  {'entities': []}),
 ('Категорія .', {'entities': []}),
 ('Графіті № 586 ( табл .', {'entities': [(0, 7, 'inscription')]}),
 ('Висота від сучасної підлоги 54 см , відстань від лівого кута 51 см . Опис .',
  {'entities': [(69, 73, 'inscription')]}),
 ('Графіті № 503 ( табл . LХІІ , 3 ) Публікації .',
  {'entities': [(0, 7, 'inscription')]}),
 ('Абетки , склади та окремі літери .', {'enti

In [8]:
import spacy
from spacy.training import Example

# https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

# Load a blank model or a pre-existing model for your language
nlp = spacy.blank('uk')  # Replace 'en' with your language code if different

# Create the NER component if it doesn't exist
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

# Add labels
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [9]:
from spacy.util import minibatch, compounding
import random
optimizer = nlp.initialize()

# Use the training data
for epoch in range(30):
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data, size=compounding(4., 32., 1.001))
    for batch in batches:
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)
    print(f"Losses at epoch {epoch}: {losses}")

Losses at epoch 0: {'ner': 972.543293432899}
Losses at epoch 1: {'ner': 356.59011461099783}


KeyboardInterrupt: 

In [10]:
def transform_to_iob(nlp_model, data):
    transformed_data = []
    for text, annot in data:
        doc = nlp_model(text)
        # Extract IOB tokens from the SpaCy doc
        iob_tokens = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_type_ else t.ent_iob_ for t in doc]
        transformed_data.append(iob_tokens)
    return transformed_data

predictions_iob = transform_to_iob(nlp, test_data)

In [11]:
predictions_iob[:3]

[['B-inscription',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-dating_criteria',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-inscription',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-inscription',
  'O',
  'B-inscription',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

In [12]:
def transform_annotations_to_iob(data, nlp):
    iob_annotations = []

    for text, annot in data:
        doc = nlp.make_doc(text)  # Create a doc from text to ensure correct offsets
        tags = ['O'] * len(doc)  # Initialize tags with 'O'

        entities = annot['entities']
        # Convert character offsets to token offsets
        for start_char, end_char, label in entities:
            start_token, end_token = None, None
            for token_index, token in enumerate(doc):
                # Check if we've found the start token
                if token.idx == start_char:
                    start_token = token_index
                # Check if the current token extends beyond the start of the end_char range
                if token.idx + len(token) == end_char:
                    end_token = token_index
                # If the token encompasses the end character
                if token.idx + len(token) > end_char and end_token is None:
                    end_token = token_index

            if start_token is not None and end_token is not None:
                tags[start_token] = f"B-{label}"
                for i in range(start_token + 1, end_token + 1):
                    tags[i] = f"I-{label}"

        iob_annotations.append(tags)
    return iob_annotations

references_iob = transform_annotations_to_iob(test_data, nlp)

In [13]:
from seqeval.metrics import classification_report, accuracy_score, f1_score

# Assuming predictions_iob and references_iob are ready
results = classification_report(references_iob, predictions_iob, digits=4)
accuracy = accuracy_score(references_iob, predictions_iob)
f1 = f1_score(references_iob, predictions_iob)

print("Classification Report:\n", results)
print("Accuracy Score:", accuracy)
print("F1 Score:", f1 * 100.0, "%")

Classification Report:
                      precision    recall  f1-score   support

    dating_criteria     0.9024    1.0000    0.9487        37
         decoration     0.7500    0.6000    0.6667        10
execution_technique     0.6000    1.0000    0.7500         9
        inscription     0.9902    1.0000    0.9951       101
           material     0.0000    0.0000    0.0000         1
           monument     0.0000    0.0000    0.0000         2
        object_type     0.0000    0.0000    0.0000         7
              other     1.0000    0.9167    0.9565        36
             symbol     0.0000    0.0000    0.0000         1

          micro avg     0.9347    0.9118    0.9231       204
          macro avg     0.4714    0.5019    0.4797       204
       weighted avg     0.8936    0.9118    0.8993       204

Accuracy Score: 0.9929595827900912
F1 Score: 92.3076923076923 %


/Users/eboros/Library/Python/3.9/lib/python/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
